In [21]:
using JuMP, Ipopt
# Solve constrained optimization 
# max( x[1] + x[2] )
# st sqrt(x[1]^2 + x[2]^2) <= 1

#Can even auto-differenitate complicated functions with embedded iterations
function squareroot(x) #pretending we don't know sqrt()
    z = x # Initial starting point for Newton’s method
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
m = Model(solver = IpoptSolver())
JuMP.register(m,:squareroot, 1, squareroot, autodiff=true) # For user defined complicated functions


@variable(m, x[1:2], start=0.5)
@objective(m, Max, sum(x))
@NLconstraint(m, squareroot(x[1]^2+x[2]^2) <= 1)
solve(m)

:Optimal

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

In [16]:
using JuMP,Ipopt
m = Model(solver = IpoptSolver())
@variable(m, x, start = 0.0)
@variable(m, y, start = 0.0)

@NLobjective(m, Min, (1-x)^2 + 100(y-x^2)^2)

solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

# adding a (linear) constraint
@constraint(m, x + y == 10)
solve(m)
println("x = ", getvalue(x), " y = ", getvalue(y))

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

In [11]:
#Auto-differentiation with ForwardDiff in Julia
using ForwardDiff
h(x) = sin(x[1]) + x[1] * x[2] + sinh(x[1] * x[2]) #multivariate.
x = [1.4 2.2]
ForwardDiff.gradient(h,x) #uses AD, seeds from x

#Or, can use complicated functions, 
f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);
g = (x) -> ForwardDiff.gradient(f, x); #New gradient function
x2 = rand(20)
g(x2) #gradient at a random 20 dim point
ForwardDiff.hessian(f,x2) #Or the hessian

#https://github.com/JuliaDiff/ReverseDiff.jl has more complicated setup


20×20 Array{Float64,2}:
 -0.622234     1.73638e-8   7.90617e-9  …   8.58784e-9   2.73281e-8
  1.73638e-8  -0.231819     1.69905e-8      1.84549e-8   5.8726e-8 
  7.90617e-9   1.69905e-8  -0.671033        8.40318e-9   2.67404e-8
  1.08178e-8   2.32476e-8   1.05852e-8      1.14978e-8   3.65881e-8
  8.51114e-9   1.82906e-8   8.32816e-9      9.04623e-9   2.87868e-8
  8.60136e-9   1.8484e-8    8.41641e-9  …   9.14182e-9   2.90904e-8
  9.16301e-8   1.96899e-7   8.96594e-8      9.73822e-8   3.09872e-7
  8.43536e-9   1.81278e-8   8.25401e-9      8.96568e-9   2.85305e-8
  1.34564e-8   2.89177e-8   1.31671e-8      1.43022e-8   4.55118e-8
  1.00337e-8   2.15626e-8   9.81798e-9      1.06645e-8   3.39363e-8
  7.93194e-9   1.70457e-8   7.7614e-9   …   8.43049e-9   2.68272e-8
  9.94346e-9   2.13687e-8   9.72969e-9      1.05686e-8   3.36311e-8
  5.35527e-8   1.15078e-7   5.2401e-8       5.69153e-8   1.81107e-7
  1.65781e-8   3.56258e-8   1.62217e-8      1.76199e-8   5.60689e-8
  4.02472e-8   8.64868e-